In [47]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [75]:
finals = pd.read_csv('semi_and_final_features.csv')
finals.columns = ['ACCOUNT_ID', 'EVENT_ID', 'TRANSACTION_COUNT', 'STATUS_ID',
       'INPLAY_BET', 'AVG_BET_SIZE', 'MAX_BET_SIZE', 'MIN_BET_SIZE',
       'STDEV_BET_SIZE']

In [172]:
finals1 = finals.loc[(finals.STATUS_ID == 'S').values]

In [173]:
finals1.loc[:, 'total'] = finals1.TRANSACTION_COUNT * finals1.AVG_BET_SIZE
finals1_by_account_event = finals1.groupby(['ACCOUNT_ID', 'EVENT_ID'])
tran_freq = finals1_by_account_event['TRANSACTION_COUNT'].sum()
bet_size = finals1_by_account_event['total'].sum()
tran_freq.name = 'tran_freq/game'
bet_size.name = 'bet_size/game'
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()
features2 = pd.merge(tran_freq, bet_size, on=['ACCOUNT_ID', 'EVENT_ID'])
features_by_account = features2.groupby('ACCOUNT_ID')
tran_freq = features_by_account['tran_freq/game'].mean()
bet_size = features_by_account['bet_size/game'].mean()
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()
features2 = pd.merge(tran_freq, bet_size, on=['ACCOUNT_ID'])

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [176]:
features2.head()

,ACCOUNT_ID,tran_freq/game,bet_size/game
0,1000002,42.333333,6742.090950
2,1000004,1.000000,943.100000
3,1000005,6.500000,234.149450
5,1000010,7.500000,900.377670
8,1000014,1.000000,181.593333


In [5]:
submit = pd.read_csv('sample_submission_bet_size.csv')
submit.columns = ['Account_ID', 'Prediction']

In [174]:
# take only the subset for submit
features2 = features2.loc[features2.ACCOUNT_ID.isin(submit.Account_ID)]

In [7]:
import glob

In [8]:
filenames = glob.glob('Datathon*.csv')

In [9]:
pre_dat = pd.DataFrame({})
for ind, filename in enumerate(filenames):
    print(ind)
    tmp_dat = pd.read_csv(filename)
    pre_dat = pre_dat.append(tmp_dat)    
pre_dat.columns = ['BET_ID', 'BET_TRANS_ID', 'MATCH_BET_ID', 'ACCOUNT_ID',
       'COUNTRY_OF_RESIDENCE_NAME', 'PARENT_EVENT_ID', 'EVENT_ID', 'MATCH',
       'EVENT_NAME', 'EVENT_DT', 'OFF_DT', 'BID_TYP', 'STATUS_ID',
       'PLACED_DATE', 'TAKEN_DATE', 'SETTLED_DATE', 'CANCELLED_DATE',
       'SELECTION_NAME', 'PERSISTENCE_TYPE', 'BET_PRICE', 'PRICE_TAKEN',
       'INPLAY_BET', 'BET_SIZE', 'PROFIT_LOSS']
peek_dat = pd.read_csv('DSMDatathon_samplefile.txt', sep='\t')
pre_dat = peek_dat.append(pre_dat)

0
1

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



2
3
4


In [10]:
pre_dat.loc[:, 'BID_TYP'] = pre_dat.BID_TYP.str.strip(' ')
pre_dat.loc[:, 'STATUS_ID'] = pre_dat.STATUS_ID.str.strip(' ')
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'correct_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'correct_profit_loss'] = - pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'correct_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'correct_profit_loss'] = - pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [66]:
label_dat = pre_dat.loc[pre_dat.EVENT_ID.isin([101183237, 101183757, 101183885, 101184013])]
label_dat = label_dat.loc[(label_dat.STATUS_ID == 'S').values]
labels = label_dat.groupby('ACCOUNT_ID')['correct_profit_loss'].sum()
labels = labels.apply(lambda x: int(x > 0))
labels = labels.reset_index()

In [179]:
dat = pre_dat.loc[~pre_dat.EVENT_ID.isin([101183237, 101183757, 101183885, 101184013])]
# dat = pre_dat

In [180]:
dat = dat.loc[(dat.STATUS_ID == 'S').values]
dat_by_account_bet = dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])
tran_freq = dat_by_account_bet.size()
bet_size = dat_by_account_bet['BET_SIZE'].sum()
tran_freq.name = 'tran_freq/game'
bet_size.name = 'bet_size/game'
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()
features = pd.merge(tran_freq, bet_size, on=['ACCOUNT_ID', 'EVENT_ID'])
features_by_account = features.groupby('ACCOUNT_ID')
tran_freq = features_by_account['tran_freq/game'].mean()
bet_size = features_by_account['bet_size/game'].mean()
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()

In [162]:
# profit rate
dat_by_account = dat.groupby('ACCOUNT_ID')
profit = dat_by_account['correct_profit_loss'].sum()
over_bet_size = dat_by_account['BET_SIZE'].sum()
profit_rate = profit / over_bet_size
profit_rate.name = 'profit_rate'
profit_rate = profit_rate.reset_index()

In [181]:
# put everything together
features = pd.merge(tran_freq, bet_size, on='ACCOUNT_ID')
# features = pd.merge(features, profit_rate, on='ACCOUNT_ID', how='left')

In [182]:
# add labels
features = pd.merge(features, labels, on='ACCOUNT_ID', how='right')
features = features.rename(columns={'correct_profit_loss': 'label'})
features = features.fillna(0)

In [188]:
features.shape

(9003, 4)

In [168]:
models = {
    'rf': RandomForestClassifier(n_estimators=100),
    'bdt': AdaBoostClassifier(n_estimators=100)}
scores = []
for model_name, model in models.items():
    print(model_name)
    clf = model
    score = cross_val_score(clf, features[['tran_freq/game', 'bet_size/game']], 
                                             features['label'], cv=5, scoring='f1')
    scores.append(score)
scores

bdt
rf


[array([ 0.39195101,  0.47030106,  0.40869565,  0.36281588,  0.49705305]),
 array([ 0.42377622,  0.44297277,  0.43023256,  0.41791045,  0.25662778])]

In [184]:
# train the models
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(features[['tran_freq/game', 'bet_size/game']], features['label'])
bdt = AdaBoostClassifier(n_estimators=100)
bdt.fit(features[['tran_freq/game', 'bet_size/game']], features['label'])

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [138]:
# features2 added dodgy "profit_rate"
features2 = pd.merge(features2, features[['ACCOUNT_ID', 'profit_rate']], on='ACCOUNT_ID', how='left')

In [185]:
features2.head()

,ACCOUNT_ID,tran_freq/game,bet_size/game
0,1000002,42.333333,6742.090950
2,1000004,1.000000,943.100000
3,1000005,6.500000,234.149450
5,1000010,7.500000,900.377670
8,1000014,1.000000,181.593333


In [187]:
features2.isnull().any()

ACCOUNT_ID        False
tran_freq/game    False
bet_size/game     False
dtype: bool

In [144]:
features2 = features2.fillna(0)

In [199]:
rfc_pred = rfc.predict_proba(features2[['tran_freq/game', 'bet_size/game']])
rfc_pred = pd.DataFrame({'Account_ID': submit.Account_ID, 'Prediction': rfc_pred[:, 1]})

In [200]:
rfc_pred.head()

,Account_ID,Prediction
0,1004240,0.67
1,1009742,0.19
2,1003908,0.36
3,1020888,0.39
4,1010074,0.16


In [201]:
rfc_pred.to_csv('rfc.csv', index=False)

In [196]:
bdt_pred = bdt.predict_proba(features2[['tran_freq/game', 'bet_size/game']])
bdt_pred = pd.DataFrame({'Account_ID': submit.Account_ID, 'Prediction': bdt_pred[:, 1]})

In [197]:
bdt_pred.head()

,Account_ID,Prediction
0,1004240,0.498705
1,1009742,0.501462
2,1003908,0.497959
3,1020888,0.498164
4,1010074,0.500636


In [198]:
bdt_pred.to_csv('bdt.csv', index=False)